In [7]:
NODE_COUNT = 8
LOG_DIR = f"/home/<User>/repos/csg_is/calculating_scores/DFL/star/fedstellar_DFL_18_07_2024_18_30_38/metrics/participant_"
date_str = "18/07/2024 16:30:38	" # minus two hours for timezone

In [8]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(NODE_COUNT):
    # Directory containing your event files
    log_dir = LOG_DIR +f"{i}"
    print(log_dir)

    date_obj = datetime.strptime(date_str.rstrip(" ").rstrip("	"), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

/home/<User>/repos/csg_is/calculating_scores/DFL/star/fedstellar_DFL_18_07_2024_18_30_38/metrics/participant_0
   step      value                     timestamp                    tag
0  2692  46.799999 2024-07-18 16:40:52.183186176  Resources/CPU_percent
1  3316  15.800000 2024-07-18 16:41:02.971129856  Resources/CPU_percent
2  2692   0.000000 2024-07-18 16:40:52.183220224     Resources/CPU_temp
3  3316   0.000000 2024-07-18 16:41:02.971172608     Resources/CPU_temp
4  2692  36.099998 2024-07-18 16:40:52.183233536  Resources/RAM_percent
2024-07-18 16:45:06 2024-07-18 16:31:02
                step     value                     timestamp
tag                                                         
Test/Accuracy   5013  0.807319 2024-07-18 16:44:59.849280000
Test/F1Score    5013  0.779649 2024-07-18 16:44:59.849312256
Test/Loss       5013  0.698691 2024-07-18 16:44:59.849261824
Test/Precision  5013  0.799914 2024-07-18 16:44:59.849291520
Test/Recall     5013  0.807319 2024-07-18 16:44:59.

In [9]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,5013,0.807319,2024-07-18 16:44:59.849280000,participant_0
Test/F1Score,5013,0.779649,2024-07-18 16:44:59.849312256,participant_0
Test/Loss,5013,0.698691,2024-07-18 16:44:59.849261824,participant_0
Test/Precision,5013,0.799914,2024-07-18 16:44:59.849291520,participant_0
Test/Recall,5013,0.807319,2024-07-18 16:44:59.849301248,participant_0
Test/Accuracy,5003,0.873544,2024-07-18 16:44:58.178713088,participant_1
Test/F1Score,5003,0.843584,2024-07-18 16:44:58.178747392,participant_1
Test/Loss,5003,1.345501,2024-07-18 16:44:58.178676736,participant_1
Test/Precision,5003,0.857162,2024-07-18 16:44:58.178725376,participant_1


In [10]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_44534/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,5015.875,0.8350
Test/F1Score,5015.875,0.8063
Test/Loss,5015.875,0.8404
Test/Precision,5015.875,0.8304
Test/Recall,5015.875,0.8350


In [11]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_44534/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,8.25,0.05
Test/F1Score,8.25,0.05
Test/Loss,8.25,0.34
Test/Precision,8.25,0.05
Test/Recall,8.25,0.05


In [12]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]

/tmp/ipykernel_44534/1227319542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_results = concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,5015.875,0.8350
Test/F1Score,5015.875,0.8063
Test/Loss,5015.875,0.8404
Test/Precision,5015.875,0.8304
Test/Recall,5015.875,0.8350
